This is a simple illustration of a [Particle Swarm](https://en.wikipedia.org/wiki/Particle_swarm_optimization) trying to find the minimum of the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function). Obviously the code is imperfect, but this should serve as a useful illustration.

Parameters can be set in the "PSO" function in the last cell of this notebook. Consider the following tasks:
- Higher search space dimensions (you may need more time in this case(
- change the population size (does a larger swarm imply shorter search time?>)
- Try out other parameter values
    - increase or decreast any of w, a1 or a2
    - change sign of w
- Implement a different goal function, e.g. the [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function). 
- Add an avarage for statistical evaluation (like wedid in the GA notebook)
- If you know python well, you can try to include an animation, e.g. to visualise search biasses 

In [1]:
import numpy as np
import itertools

The following will be the goal ("fitness") function. Here it is to be minimised.

In [2]:
def rosenbrock(pos,dim):    # this serves as a goal function
                            # Defined by f(x,y) = (a-x)^2 + b(y-x^2)^2
                            # Using here: a = 1, b= 100, optimum 0 at (1,1)
        if dim==2:
            return ((1-pos[0])**2 + 100*(pos[1] - pos[0]**2)**2)
        elif dim==1:
            return (1-pos[0])**2 
        else: 
            ros=0;
            for i in range(dim-1):
                ros=ros+100*(pos[i+1] - pos[i]**2)**2 * (1-pos[i])**2 
            return ros

In [3]:
def f1(pos, dim):
    return sum([x**2 for x in pos])

def f2(pos, dim):
    return 10*dim + sum([(x**2 - 10*np.cos(2*np.pi*x)) for x in pos])

In [4]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx,algo = 'r'):
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim
        self.algo = algo
 

        self.fitness = self.getfitness(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos,algo = 'r'):
        self.position = pos
        self.fitness = self.getfitness(self.position,self.dim)
        
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos
      
            
    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        
        a1r1 = np.multiply(a1, r1)
        
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level 
        return inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
    
    def updateVel_repulsive(self, inertia, a1, a2, a3, best_self_pos, best_swarm_pos, poses,idx):
        classic_terms = self.updateVel(inertia, a1, a2, best_self_pos, best_swarm_pos)
        r3 = np.random.uniform(low=0, high=1, size = self.dim)
        a3r3 = np.multiply(a3, r3)
        rep = 0
        for i in range(len(poses)):
            if not i == idx:
                dist = sum([x**2 for x in np.subtract(poses[i].position, self.position)])
                rep += np.subtract(poses[i].position, self.position)/(dist+np.random.uniform(low=0, high=0.2)**2) 
        return classic_terms - np.multiply(a3r3, rep)/(len(poses)-1)
    
    def getfitness(self,pos,dim):
        if self.algo == 'f1':
            self.fitness = f1(self.position,dim)
        elif self.algo == 'f2':
            self.fitness = f2(self.position,dim)
        else:
            self.fitness = rosenbrock(self.position,dim)
        return self.fitness
    

In [10]:
class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim, population_size, time_steps, search_range,algo='r',rep=0,a3=0):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a2 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.a3 = a3
        self.dim = dim
        self.rep = rep

        self.swarm = [Particle(dim,-search_range,search_range,algo) for i in range(population_size)]
        self.time_steps = time_steps
        #print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-5.12, high=5.12, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self):
        for t in range(self.time_steps):
            #print(t)
            for p in range(len(self.swarm)):
                particle = self.swarm[p]
                #print(particle.position)
                if self.rep == 0:
                    new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                else:
                    new_position = particle.position + particle.updateVel_repulsive(self.w, self.a1, self.a2, self.a3,
                                                                                    particle.best_particle_pos, self.best_swarm_pos,poses=self.swarm,idx=p)
                #print(new_position)
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')

                
                self.swarm[p].setPos(self.posWithinRange(new_position))
                new_fitness = self.swarm[p].getfitness(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position

            if t % 10 == 0: #we print only two components even it search space is high-dimensional
                if self.dim == 2:
                    print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %9.4f,%9.4f\n" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ") 
                if self.dim == 3:
                    print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %9.4f,%9.4f,%9.4f\n" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1],self.best_swarm_pos[2]), end =" ") 
                elif self.dim>3: 
                    print('...')
                else:
                    print('')
                    
    def posWithinRange(self,pos):
        new_position = pos
        if new_position[0] > 5.12:
            new_position[0] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[0] < -5.12:
            new_position[0] = -5.12+np.random.uniform(low=0, high=1)

        if new_position[1] > 5.12:
            new_position[1] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[1] < -5.12:
            new_position[1] = -5.12+np.random.uniform(low=0, high=1)
        return new_position
              

Standard values are dim=2, w=0.7, a1=2.02, a2=2.02, population_size=30; but feel free to try others ones.

In [6]:
PSO(dim=2, w=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=101, search_range=5.12,algo='f1').run()

Time:      0,  Best Fitness:    1.154781537,  Best Pos:   -1.0201,   0.3378 
Time:     10,  Best Fitness:    0.000002340,  Best Pos:    0.0014,   0.0006 
Time:     20,  Best Fitness:    0.000002340,  Best Pos:    0.0014,   0.0006 
Time:     30,  Best Fitness:    0.000001346,  Best Pos:    0.0011,   0.0003 
Time:     40,  Best Fitness:    0.000001346,  Best Pos:    0.0011,   0.0003 
Time:     50,  Best Fitness:    0.000001346,  Best Pos:    0.0011,   0.0003 
Time:     60,  Best Fitness:    0.000000310,  Best Pos:    0.0003,  -0.0004 
Time:     70,  Best Fitness:    0.000000310,  Best Pos:    0.0003,  -0.0004 
Time:     80,  Best Fitness:    0.000000310,  Best Pos:    0.0003,  -0.0004 
Time:     90,  Best Fitness:    0.000000310,  Best Pos:    0.0003,  -0.0004 
Time:    100,  Best Fitness:    0.000000310,  Best Pos:    0.0003,  -0.0004 


In [7]:
PSO(dim=2, w=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=1001, search_range=5.12,algo='f2').run()

Time:      0,  Best Fitness:    5.530334944,  Best Pos:   -2.0325,   0.9634 
Time:     10,  Best Fitness:    2.265638194,  Best Pos:   -0.9446,   0.0628 
Time:     20,  Best Fitness:    0.044353776,  Best Pos:   -0.0034,   0.0146 
Time:     30,  Best Fitness:    0.007855072,  Best Pos:   -0.0032,  -0.0054 
Time:     40,  Best Fitness:    0.000643361,  Best Pos:   -0.0002,  -0.0018 
Time:     50,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:     60,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:     70,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:     80,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:     90,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:    100,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:    110,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 
Time:    120,  Best Fitness:    0.000448234,  Best Pos:    0.0009,   0.0012 

In [11]:
PSO(dim=3, w=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=101, search_range=5.12,algo='f1').run()

Time:      0,  Best Fitness:    0.299173379,  Best Pos:   -0.1039,  -0.3622,  -0.3965
 Time:     10,  Best Fitness:    0.000367196,  Best Pos:    0.0161,   0.0057,   0.0087
 Time:     20,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     30,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     40,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     50,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     60,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     70,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     80,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:     90,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 Time:    100,  Best Fitness:    0.000013303,  Best Pos:   -0.0032,   0.0005,   0.0017
 

In [12]:
PSO(dim=3, w=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=1001, search_range=5.12,algo='f2').run()

Time:      0,  Best Fitness:    9.825761598,  Best Pos:   -0.9126,  -1.0014,  -1.8732
 Time:     10,  Best Fitness:    3.208335455,  Best Pos:   -0.9669,  -0.9786,   0.9861
 Time:     20,  Best Fitness:    2.723523133,  Best Pos:   -0.0398,  -0.9566,   0.9693
 Time:     30,  Best Fitness:    2.006759394,  Best Pos:    0.0029,  -0.9921,   1.0032
 Time:     40,  Best Fitness:    2.006759394,  Best Pos:    0.0029,  -0.9921,   1.0032
 Time:     50,  Best Fitness:    1.142236141,  Best Pos:   -0.0165,  -0.0114,   1.0135
 Time:     60,  Best Fitness:    1.043237590,  Best Pos:   -0.0000,  -0.0145,   1.0008
 Time:     70,  Best Fitness:    1.024660113,  Best Pos:    0.0076,  -0.0041,   0.9863
 Time:     80,  Best Fitness:    1.007961057,  Best Pos:   -0.0010,   0.0072,   0.9984
 Time:     90,  Best Fitness:    1.007961057,  Best Pos:   -0.0010,   0.0072,   0.9984
 Time:    100,  Best Fitness:    0.069933615,  Best Pos:    0.0129,  -0.0062,   0.0122
 Time:    110,  Best Fitness:    0.038187042

In [15]:
PSO(dim=2, w=0.6, a1=1.02, a2=0.82, population_size=30, time_steps=101, search_range=5.12,algo='f1',rep=1,a3=0.5).run()

Time:      0,  Best Fitness:    0.053767438,  Best Pos:    0.0863,  -0.2152
 
Time:     10,  Best Fitness:    0.002769103,  Best Pos:    0.0375,   0.0369
 
Time:     20,  Best Fitness:    0.000282629,  Best Pos:    0.0016,   0.0167
 
Time:     30,  Best Fitness:    0.000282629,  Best Pos:    0.0016,   0.0167
 
Time:     40,  Best Fitness:    0.000282629,  Best Pos:    0.0016,   0.0167
 
Time:     50,  Best Fitness:    0.000282629,  Best Pos:    0.0016,   0.0167
 
Time:     60,  Best Fitness:    0.000030500,  Best Pos:    0.0007,   0.0055
 
Time:     70,  Best Fitness:    0.000030500,  Best Pos:    0.0007,   0.0055
 
Time:     80,  Best Fitness:    0.000030500,  Best Pos:    0.0007,   0.0055
 
Time:     90,  Best Fitness:    0.000030500,  Best Pos:    0.0007,   0.0055
 
Time:    100,  Best Fitness:    0.000030500,  Best Pos:    0.0007,   0.0055
 


In [16]:
PSO(dim=2, w=0.7, a1=1.02, a2=1.02, population_size=30, time_steps=101, search_range=5.12,algo='f1').run()

Time:      0,  Best Fitness:    0.046714432,  Best Pos:    0.1003,   0.1915
 
Time:     10,  Best Fitness:    0.000279173,  Best Pos:   -0.0148,  -0.0077
 
Time:     20,  Best Fitness:    0.000009579,  Best Pos:    0.0026,   0.0017
 
Time:     30,  Best Fitness:    0.000008031,  Best Pos:    0.0024,   0.0016
 
Time:     40,  Best Fitness:    0.000008020,  Best Pos:    0.0028,   0.0002
 
Time:     50,  Best Fitness:    0.000008020,  Best Pos:    0.0028,   0.0002
 
Time:     60,  Best Fitness:    0.000008020,  Best Pos:    0.0028,   0.0002
 
Time:     70,  Best Fitness:    0.000005767,  Best Pos:    0.0023,  -0.0008
 
Time:     80,  Best Fitness:    0.000005767,  Best Pos:    0.0023,  -0.0008
 
Time:     90,  Best Fitness:    0.000005767,  Best Pos:    0.0023,  -0.0008
 
Time:    100,  Best Fitness:    0.000005767,  Best Pos:    0.0023,  -0.0008
 


0.2279804688761805